## Regresión logística

<div style="text-align: justify"> 
La regresión logística a diferencia de la regresión lineal no busca predecir un valor numérico, sino que determina la probabilidad de pertenecer de una categoría de acuerdo a los valores de las variables independientes. Otro elemento que diferencia a estas dos regresiones es el supuesto de normalidad, donde la regresión logística estima la probabilidad de ocurrencia de un acontecimiento siguiendo una distribución de probabilidad binomial.
</div>
    
$$odds=\frac{\text{Probabilidad de éxito}}{\text{Probabilidad de fracaso}}\$$\\


<div style="text-align: justify">
La regresión logística se basa en el logaritmo natural de este cociente, como se muestra en la ecuación, en la que los valores positivos de beta aumentan la probabilidad de ocurrencia
</div>
    
$$\ln(\frac{P}{1-P})= \beta_{0} +\beta_{1}x_{1}+\beta_{2}x_{2}+\beta_{3}x_{3} $$



### Importar módulos/ paquetes

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

### Cargar y preparar conjunto de datos para ajuste del modelo de regresión

In [ ]:
r = "Automobile per gallon.csv"
datos = pd.read_csv(r,sep=",",index_col=0)

# Nombre de las variables dependientes e independientes
nombres = ['mpg', 'cyl', 'displ', 'hp', 'weight', 'accel', 'yr', 'size',"origin"]  

In [35]:
datos.info

,mpg,cyl,displ,hp,weight,accel,yr,size
count,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000
mean,23.445918,5.471939,194.411990,104.469388,2977.584184,15.541327,75.979592,26.626810
std,7.805007,1.705783,104.644004,38.491160,849.402560,2.758864,3.683737,15.208310
min,9.000000,3.000000,68.000000,46.000000,1613.000000,8.000000,70.000000,7.227136
25%,17.000000,4.000000,105.000000,75.000000,2225.250000,13.775000,73.000000,13.754831
50%,22.750000,4.000000,151.000000,93.500000,2803.500000,15.500000,76.000000,21.832290
75%,29.000000,8.000000,275.750000,126.000000,3614.750000,17.025000,79.000000,36.295630
max,46.600000,8.000000,455.000000,230.000000,5140.000000,24.800000,82.000000,73.387778


In [ ]:
X = datos.drop(["name","color","marker","origin"],axis=1).values    # Se especifican las variables independientes
y = datos["origin"].values                  # Se especifica la variable dependiente

# Se realiza la partición del conjunto de observaciones en un subconjunto para prueba y otro para
# entrenamiento. En este caso 80/20

# Particionar datos de prueba y entrenamiento
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3)

In [ ]:
# Como parte del pre-procesamiento se escalan o normalizan las observaciones 

scaler = preprocessing.StandardScaler().fit(X_train) 

X_train_scaled = scaler.transform(X_train) # Se escalan las observaciones de entrenamiento

X_test_scaled = scaler.transform(X_test)   # Se escalan las observaciones de prueba

### Entrenamiento del modelo

In [ ]:
logreg = LogisticRegression(max_iter=100) 
logreg.fit(X_train_scaled,y_train)        # Se ajusta el modelo con los datos de entrenamiento

In [ ]:
# Se crea un data frame para visualizar los datos para cada regresión 

coef = pd.DataFrame(logreg.coef_, columns=nombres[:-1])
print("Coeficientes de regresión: \n")
coef

### Prueba y evaluación de desempeño del modelo

In [ ]:
predict = logreg.predict(X_test_scaled)   # Pronósticar nuevas observaciones. Se usan datos de prueba

print(confusion_matrix(y_test,predict))       # Imprimir la matriz de confusión
print(classification_report(y_test,predict))  # Imprimir un reporte de clasificación

In [ ]:
plt.hist([y_test,predict],label=["Valor pronosticado","Valor real"]) # Histograma de valores pronosticados
plt.title("Valor observado vs pronóstico")
plt.ylabel('Frecuencia')
plt.xlabel("Clase")
plt.legend(loc=0)
plt.show()

### Selección de características

In [32]:
rfe = RFE(logreg)
ref = rfe.fit(X_train_scaled,y_train)
car = pd.DataFrame(ref.ranking_,index=nombres[:-1],columns=["Importancia"])
print("Listado de las variables:")
print(car.sort_values(by="Importancia"))

Listado de las variables:
        Importancia
displ             1
hp                1
weight            1
size              1
mpg               2
yr                3
cyl               4
accel             5


Incluyendo las variable identificadas como influyentes

In [ ]:
Best = ref.ranking_ == 1  # Se seleccionan las variables consideradas como las mejores

K=[]                      # Se crea una variable para almacenar las que fueron identificadas como las mejores
for i in range(0,len(X)):
    K.append(X[i][Best])
X_new = np.asarray(K)

X_train2,X_test2 = train_test_split(X_new,test_size=0.2, random_state=1234)
scaler2 = preprocessing.StandardScaler().fit(X_train2)
X_train_scaled2 = scaler2.transform(X_train2)
X_test_scaled2 = scaler2.transform(X_test2)
X_test_scaled2

# Predicción con los mejores

logreg2 = LogisticRegression()
logreg2.fit(X_train_scaled2,y_train)

predict2=logreg2.predict(X_test_scaled2)
print(confusion_matrix(y_test,predict2))
print(classification_report(y_test,predict2))

In [ ]:
plt.hist([y_test,predict2],label=["Valor pronosticado","Valor real"])
plt.title("Valor observado vs pronóstico")
plt.ylabel('error')
plt.legend(loc=0)
plt.show()